# 1. Layers and Blocks

In [2]:
import torch 
import torch.nn as nn
from torch.nn import functional as F

## Blocks

![](http://d2l.ai/_images/blocks.svg)

In [14]:
X = torch.rand(2, 20)
X

tensor([[0.5939, 0.6175, 0.1714, 0.5397, 0.1526, 0.4353, 0.1990, 0.5693, 0.9766,
         0.0524, 0.6760, 0.6836, 0.6711, 0.0801, 0.9362, 0.4423, 0.0635, 0.9489,
         0.4270, 0.1375],
        [0.9727, 0.6019, 0.7009, 0.0513, 0.4568, 0.0521, 0.7973, 0.3518, 0.9398,
         0.8829, 0.0683, 0.3902, 0.7581, 0.8967, 0.6151, 0.3569, 0.1090, 0.6570,
         0.5716, 0.2735]])

In [15]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1395, -0.0907, -0.0338,  0.1819,  0.0842,  0.1264,  0.2354, -0.0636,
          0.1333,  0.1141],
        [-0.0733, -0.0031, -0.0956,  0.0419,  0.0919, -0.1337,  0.0886, -0.1173,
          0.1903,  0.1289]], grad_fn=<AddmmBackward0>)

We can define an equivalent MLP class s a sub-class the inherents the nn.Module class:

In [16]:
class MLP(nn.Module): 
    
    def __init__(self):
        super().__init__()       
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [17]:
net = MLP()
net(X)

tensor([[ 0.0509, -0.0054, -0.0512, -0.0098, -0.0104, -0.1588, -0.0423, -0.1558,
          0.1375,  0.1957],
        [ 0.0320, -0.0381,  0.0607, -0.0225, -0.0079, -0.0780, -0.0176, -0.2375,
          0.0732,  0.1693]], grad_fn=<AddmmBackward0>)

We can also define a sequential class:

In [18]:
class MySequential(nn.Module):
    
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [19]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1916,  0.2378,  0.0630,  0.0474, -0.1220, -0.0640, -0.1546, -0.0257,
          0.0176,  0.2582],
        [ 0.1391,  0.2432,  0.0060,  0.1216, -0.0837,  0.0196, -0.0626,  0.0132,
          0.0541,  0.2981]], grad_fn=<AddmmBackward0>)

## Self-Define Layers

We define a layer that inherent the nn.Module class and just change the forward method:

In [46]:
class CenteredLayer(nn.Module):
    
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [47]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [48]:
net = nn.Sequential(nn.Linear(8,128), 
                    CenteredLayer())
net(torch.rand(4,8)).mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

We can also define layers with parameters:

In [49]:
class MyLinear(nn.Module):
    
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.zeros(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [50]:
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[-0.2683,  0.3553,  1.1462],
        [-0.7788,  0.1474,  0.1835],
        [-0.8980,  0.5986,  0.5469],
        [ 1.2813, -0.5565,  0.8941],
        [-1.7249,  0.3526, -0.1716]], requires_grad=True)

In [51]:
linear(torch.rand(2, =5))

tensor([[0.0000, 0.5389, 1.8235],
        [0.0000, 0.1976, 1.9796]])

In [52]:
net = nn.Sequential(MyLinear(64,8), 
                    CenteredLayer(),
                    MyLinear(8,1))
net(torch.rand(2,64))

tensor([[1.9500],
        [9.2871]])